In [1]:
import pandas as pd

# Calculate population under 1.9\$ per day for each income groups

This is to show how many people in higher income groups are actually poor people.

use WB's income group when it's available, otherwise use GM's income group

In [8]:
# gm groups:
# TODO: move to fixtures
gm_groups = pd.read_csv('../../../ddf--gapminder--fasttrack/ddf--datapoints--ilevels4--by--country--time.csv')
# wb groups:
wb_groups = pd.read_csv('../source/fixtures/wb_income_groups.csv')

In [9]:
gm_groups['ilevels4'].unique()

array(['Level 1', 'Level 2', 'Level 3', 'Level 4'], dtype=object)

In [10]:
wb_groups

,geo,name,time,4 income groups (WB),3 income groups (WB)
0,afg,Afghanistan,1987,Low income,Low income
1,afg,Afghanistan,1988,Low income,Low income
2,afg,Afghanistan,1989,Low income,Low income
3,afg,Afghanistan,1990,Low income,Low income
4,afg,Afghanistan,1991,Low income,Low income
...,...,...,...,...,...
6659,zwe,Zimbabwe,2016,Low income,Low income
6660,zwe,Zimbabwe,2017,Low income,Low income
6661,zwe,Zimbabwe,2018,Lower middle income,Middle income
6662,zwe,Zimbabwe,2019,Lower middle income,Middle income


In [11]:
wb_groups['4 income groups (WB)'].unique()

array(['Low income', 'Lower middle income', 'Upper middle income',
       'High income'], dtype=object)

In [12]:
wb_groups = wb_groups[['geo', 'time', '4 income groups (WB)']].copy()
wb_groups.columns = ['geo', 'time', 'income_groups']

In [14]:
m1 = { "Level 1": 'lic', 'Level 2': 'lmic', 'Level 3': 'umic', 'Level 4': 'hic'}
m2 = {'Low income': 'lic', 'Lower middle income': 'lmic', 'Upper middle income': 'umic', 'High income': 'hic'}

In [15]:
gm_groups['ilevels4'] = gm_groups['ilevels4'].map(lambda x: m1[x])
wb_groups['income_groups'] = wb_groups['income_groups'].map(lambda x: m2[x])

In [32]:
df1 = gm_groups.set_index(['country', 'time'])['ilevels4']
df2 = wb_groups.set_index(['geo', 'time'])['income_groups']

In [33]:
df1.index.names = ['country', 'time']
df2.index.names = ['country', 'time']

df1.name = 'ilevel4'
df2.name = 'ilevel4'

In [34]:
ilevel4, ilevel4_wb = df1.align(df2)

In [35]:
# start with gapminder's groups, then update with wb.
ilevel4.update(ilevel4_wb)

In [36]:
ilevel4

country  time
afg      1800     lic
         1801     lic
         1802     lic
         1803     lic
         1804     lic
                 ... 
zwe      2046    lmic
         2047    lmic
         2048    lmic
         2049    lmic
         2050    lmic
Name: ilevel4, Length: 48979, dtype: object

In [37]:
ilevel4.hasnans

False

In [47]:
ilevel4[~ilevel4.index.isin(df1.index.values)]  # this is wb's data which is not found in gm.

country  time
lie      1987    hic
         1988    hic
         1989    hic
         1990    hic
         1991    hic
         1992    hic
         1993    hic
         1994    hic
         1995    hic
         1996    hic
         1997    hic
         1998    hic
         1999    hic
         2000    hic
         2001    hic
         2002    hic
         2003    hic
         2004    hic
         2005    hic
         2006    hic
         2007    hic
         2008    hic
         2009    hic
         2010    hic
         2011    hic
         2012    hic
         2013    hic
         2014    hic
         2015    hic
         2016    hic
         2017    hic
         2018    hic
         2019    hic
         2020    hic
Name: ilevel4, dtype: object

In [45]:
ilevel4[~ilevel4.index.isin(df1.index.values)].shape

(34,)

In [46]:
48945 - 48979

-34

In [48]:
df = pd.read_csv('../../poverty_rates/ddf--datapoints--poverty_population--by--country--time--wb_level.csv')

In [49]:
df.head()

,country,time,wb_level,poverty_population
0,afg,1800,wb_lvl1,3017073
1,afg,1800,wb_lvl2,178027
2,afg,1800,wb_lvl3,57702
3,afg,1800,wb_lvl4,18868
4,afg,1800,wb_lvl5,8330


In [50]:
df1 = df[df.wb_level == 'wb_lvl1']  # <1.9$/day

In [51]:
df1 = df1[['country', 'time', 'poverty_population']].set_index(['country', 'time'])

In [52]:
df1

poverty_population
country time                    
afg     1800             3017073
        1801             3017073
        1802             3017073
        1803             3017073
        1804             3017073
...                          ...
zwe     2046             5484586
        2047             5357587
        2048             5229870
        2049             5102145
        2050             4970076

[48945 rows x 1 columns]

In [55]:
ilevel4.to_frame()

ilevel4
country time        
afg     1800     lic
        1801     lic
        1802     lic
        1803     lic
        1804     lic
...              ...
zwe     2046    lmic
        2047    lmic
        2048    lmic
        2049    lmic
        2050    lmic

[48979 rows x 1 columns]

In [59]:
df2 = pd.concat([df1, ilevel4.to_frame()], axis=1)

In [57]:
df2

poverty_population ilevel4
country time                            
afg     1800           3017073.0     lic
        1801           3017073.0     lic
        1802           3017073.0     lic
        1803           3017073.0     lic
        1804           3017073.0     lic
...                          ...     ...
lie     2016                 NaN     hic
        2017                 NaN     hic
        2018                 NaN     hic
        2019                 NaN     hic
        2020                 NaN     hic

[48979 rows x 2 columns]

In [60]:
res = df2.reset_index()

In [61]:
res = res.dropna(subset=['poverty_population'])

In [62]:
res = res.groupby(['ilevel4', 'time'])['poverty_population'].sum()

In [63]:
res

ilevel4  time
hic      1944          63.0
         1945          62.0
         1946          45.0
         1947          26.0
         1948          15.0
                    ...    
umic     2046    65180611.0
         2047    63563618.0
         2048    61958075.0
         2049    55264193.0
         2050    53978007.0
Name: poverty_population, Length: 774, dtype: float64

In [64]:
res = res.reset_index()

In [65]:
gs = res['ilevel4'].unique()

In [66]:
gs

array(['hic', 'lic', 'lmic', 'umic'], dtype=object)

In [67]:
gt = pd.CategoricalDtype(['lic', 'lmic', 'umic', 'hic'], ordered=True)

In [68]:
res['ilevel4'] = res['ilevel4'].astype(gt)

In [69]:
res

,ilevel4,time,poverty_population
0,hic,1944,63.0
1,hic,1945,62.0
2,hic,1946,45.0
3,hic,1947,26.0
4,hic,1948,15.0
...,...,...,...
769,umic,2046,65180611.0
770,umic,2047,63563618.0
771,umic,2048,61958075.0
772,umic,2049,55264193.0


In [71]:
res = res.sort_values(by=['ilevel4', 'time'])

In [72]:
res.columns = ['income_groups', 'time', 'poverty_population_under_1_9_per_day']

In [73]:
res

,income_groups,time,poverty_population_under_1_9_per_day
107,lic,1800,758276971.0
108,lic,1801,761135220.0
109,lic,1802,764406770.0
110,lic,1803,767712908.0
111,lic,1804,771031396.0
...,...,...,...
102,hic,2046,7913896.0
103,hic,2047,7607320.0
104,hic,2048,7262310.0
105,hic,2049,12323380.0


In [74]:
m = {'lic': 'low_income', 'hic': 'high_income', 'lmic': 'lower_middle_income', 'umic': 'upper_middle_income'}

In [75]:
res['income_groups'] = res['income_groups'].map(m)

In [76]:
res

,income_groups,time,poverty_population_under_1_9_per_day
107,low_income,1800,758276971.0
108,low_income,1801,761135220.0
109,low_income,1802,764406770.0
110,low_income,1803,767712908.0
111,low_income,1804,771031396.0
...,...,...,...
102,high_income,2046,7913896.0
103,high_income,2047,7607320.0
104,high_income,2048,7262310.0
105,high_income,2049,12323380.0


In [77]:
res.dtypes['income_groups']

CategoricalDtype(categories=['low_income', 'lower_middle_income', 'upper_middle_income',
                  'high_income'],
, ordered=True)

In [78]:
res.to_csv('../../poverty_rates/ddf--datapoints--poverty_population_under_1_9_per_day--by--income_groups--time.csv', index=False)